### Here I create BED files for each species comparison.

In [18]:
setwd("/Users/rebecca/sudmant/analyses/myotis/analysis/reciprocal_best_hits/01_get_best_hits")

library(pafr)
library(dplyr, quietly = TRUE)

options(scipen = 100, digits = 10)

myo_meta <- read.csv("../../../data/myotis_meta.csv")
myo_meta

Field_Name,Abbr
<chr>,<chr>
Myotis-Auriculus,mMyoAui
Myotis-Californicus,mMyoCai
Myotis-Evotis,mMyoEvo
Myotis-Lucifugus,mMyoLuc
Myotis-Occultus,mMyoOcc
Myotis-Thysanodes,mMyoThy
Myotis-Velifer,mMyoVel
Myotis-Volans,mMyoVol
Myotis-Yumanensis,mMyoYum


In [19]:
for (i in seq_along(myo_meta$Abbr)) {
  
  for (j in seq_along(myo_meta$Abbr)) {

    if (i < j) {

      # working_paf1 <- read_paf(file.path("resources/pafs/trimmed", intersect(
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("^", myo_meta$Abbr[j])),
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("_", myo_meta$Abbr[i], ".*_trimmed.paf"))
      # )), include_tags = FALSE)

      # working_paf2 <- read_paf(file.path("resources/pafs/trimmed", intersect(
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("^", myo_meta$Abbr[i])),
      #   list.files(path = "resources/pafs/trimmed", pattern = paste0("_", myo_meta$Abbr[j], ".*_trimmed.paf"))
      # )), include_tags = FALSE)
      
      working_paf1 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[j])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[i]))
      )), include_tags = FALSE)

      working_paf2 <- read_paf(file.path("resources/pafs", intersect(
        list.files(path = "resources/pafs", pattern = paste0("^", myo_meta$Abbr[i])),
        list.files(path = "resources/pafs", pattern = paste0("_", myo_meta$Abbr[j]))
      )), include_tags = FALSE)

      ## Filter sequences with mapq < 10:
      ## Sniffles mapping quality threshold to 10
      ## Ref: https://arxiv.org/pdf/2108.03515.pdf
      
      # working_paf1 <- working_paf1[working_paf1$mapq >= 10,]
      # working_paf2 <- working_paf2[working_paf2$mapq >= 10,]

      ## Make unique ID for each hit:
      
      working_paf1$Id <- with(working_paf1, paste(qstart, qend, tstart, tend, sep = "_"))      
      working_paf2$Id <- with(working_paf2, paste(qstart, qend, tstart, tend, sep = "_"))      

      ## Make BEDs for species 1 positions:

      spec1_query <- working_paf1 %>%
        dplyr::select(
          qname, qstart, qend, strand,
          nmatch, alen, mapq, Id
        )

      spec1_target <- working_paf2 %>%
        dplyr::select(
          tname, tstart, tend,
          nmatch, alen, mapq, Id
        )

      write.table(
        spec1_query,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_positions_query.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

      write.table(
        spec1_target,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[i], "_vs_", myo_meta$Abbr[j], "_positions_target.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

      ## Species 2 positions:

      spec2_query <- working_paf2 %>%
        dplyr::select(
          qname, qstart, qend, strand,
          nmatch, alen, mapq, Id
        )

      spec2_target <- working_paf1 %>%
        dplyr::select(
          tname, tstart, tend,
          nmatch, alen, mapq, Id
        )

      write.table(
        spec2_query,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_positions_query.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

      write.table(
        spec2_target,
        file = paste0(
          "resources/beds/", myo_meta$Abbr[j], "_vs_", myo_meta$Abbr[i], "_positions_target.bed"
        ), sep = "\t", row.names = FALSE, col.names = FALSE, quote = FALSE
      )

    }
    
  }
  
}
